In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
from tensorflow.keras import layers
import string

2025-11-07 17:26:54.158704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762536414.357326      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762536414.411518      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
BATCH_SIZE = 64
NUM_HEADS = 12
NUM_BLOCKS = 12
EMBED_DIM = 768
DENSE_DIM = 3072
DROPOUT_RATE = 0.1
CHUNK_LENGTH = 200

In [3]:
df = pd.read_csv('/kaggle/input/the-bards-best-a-character-modeling-dataset/train.csv')

In [4]:
text = '<sos>' + df.values[0][0] + '<eos>'
text = re.sub(r'\s+', ' ', str(text)).strip()

In [5]:
def chunk_text_by_words(text, max_words):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

In [6]:
chunks = chunk_text_by_words(text, CHUNK_LENGTH+1)

In [7]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 18000
sequence_length = CHUNK_LENGTH+1

vectorizer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

vectorizer.adapt(chunks)

I0000 00:00:1762536427.047656      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [8]:
def make_dataset(chunks):
    tokens = vectorizer(chunks)
    tokens_inp = tokens[:,:CHUNK_LENGTH]
    tokens_out = tokens[:,1:]
    ds = tf.data.Dataset.from_tensor_slices((tokens_inp,tokens_out))
    ds = ds.batch(BATCH_SIZE)
    ds = ds.shuffle(1024).prefetch(16).cache()
    return ds

In [9]:
ds = make_dataset(chunks)

In [10]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim):
        super().__init__()
        self.positional_embedding = tf.keras.layers.Embedding(input_dim = sequence_length, output_dim = output_dim, mask_zero=False)
        self.token_embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim= output_dim, mask_zero=True)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.positional_embedding(positions)
        return embedded_tokens + embedded_positions

In [11]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, num_heads, embed_dim, dense_dim, dropout_rate):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                           key_dim=embed_dim//num_heads)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense_proj = tf.keras.models.Sequential([
            tf.keras.layers.Dense(dense_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    def call(self, inputs):
        attn_out = self.attention(query=inputs,
                            key=inputs,
                            value=inputs,
                            use_causal_mask=True)
        norm1_out = self.layernorm1(attn_out+inputs)
        drop1_out = self.dropout1(norm1_out)
        dense_proj_out = self.dense_proj(drop1_out)
        norm2_out = self.layernorm2(drop1_out+dense_proj_out)
        drop2_out = self.dropout2(norm2_out)
        return drop2_out

In [12]:
inputs = tf.keras.layers.Input(shape=(None,))
embeddings = PositionalEmbedding(sequence_length, vocab_size, EMBED_DIM)(inputs)
x = embeddings
for layer in range(NUM_BLOCKS):
    x = TransformerDecoder(NUM_HEADS, EMBED_DIM, DENSE_DIM, DROPOUT_RATE)(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(vocab_size,activation='softmax')(x)
transformer = tf.keras.models.Model(inputs, output)

In [13]:
transformer.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding            │ (None, None, 768)      │    13,978,368 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder             │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_2           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_3           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_4           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_5           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_6           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_7           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_8           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_9           │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_10          │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_11          │ (None, None, 768)      │     7,087,872 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, None, 768)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, None, 18000)    │    13,842,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,874,832 (430.58 MB)

 Trainable params: 112,874,832 (430.58 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
transformer.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                    metrics = ['accuracy'],
                    optimizer='adam')
transformer.fit(ds, epochs = 50)

Epoch 1/50


I0000 00:00:1762536475.439895      94 service.cc:148] XLA service 0x7e82cc01b6d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762536475.440637      94 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1762536477.848757      94 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1762536479.772882      94 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1762536506.906227      94 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 7/15 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.0173 - loss: 8.9540 

W0000 00:00:1762536518.242271      97 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


15/15 ━━━━━━━━━━━━━━━━━━━━ 116s 3s/step - accuracy: 0.0208 - loss: 8.3191
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0282 - loss: 6.9225
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0271 - loss: 6.8939
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0282 - loss: 6.8686
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0287 - loss: 6.8549
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0279 - loss: 6.8498
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0261 - loss: 6.8453
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0268 - loss: 6.8428
Epoch 9/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0277 - loss: 6.8411
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0295 - loss: 6.8403
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0297 - loss: 6.8393
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0300 - loss: 6.8338